# Setup

## Libraries and functions

In [1]:
#Import all the things!
#Utility libraries

import glob
import os
import re
import itertools as it

#Data wrangling
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import pandas.tseries.holiday as hol
from sklearn.model_selection import train_test_split

#Viz
import matplotlib.pyplot as plt
%matplotlib inline


In [162]:
def pretty_shape(my_df,desc):
    '''
    Displays the shape of a dataframe in a prettier format.
    I don't know why the default display bugs me. It just does.
    my_df: source dataframe
    desc: Descriptive text
    '''
    print(f'\n{desc}:')
    display(pd.DataFrame(my_df.shape,
                         columns=['count'])\
                         .rename(index={0:"rows",1:"cols"})\
                         .T)
    

# Convenient for illustration purposes
def display_embiggen(my_df,rows_temp=50):
    '''
    Purpose: Expand the default display for Jupyter notebook output, and revert to
             previous boring values afterward
    Requires: df: input dataframe/Series for display
              rows: number of rows to display (default 50)
    Warning: Managenent is not responsible for ugly notebook output if you forget to 
             set the cell output to scroll 
    '''
    with pd.option_context("display.min_rows", rows_temp, "display.max_rows", rows_temp):
        display(my_df)

## Weather data import

In [66]:
weather_files=glob.glob('../data/en_climate_hourly_BC_1108395*.csv')
weather_data_raw=pd.concat([pd.read_csv(eccc_dump) for eccc_dump in weather_files])
#Doesn't need to be pretty, but we don't need spaces and parens in our column headers
weather_data_raw.rename(columns=lambda x: re.sub('\W+','',x),inplace=True)
weather_data_raw.rename(columns={'DateTime':'Timestamp'},inplace=True)


## Trip data import

In [4]:
def xform_columns(cols):
    '''
    Rename columns from Mobi data files.  Not always super-consistent
    '''
    my_col_map={}
    for my_col in cols:
        my_col_map[my_col]=re.sub('\s+','_',re.sub('\s+\(.*\)','',my_col)).lower()
        #Handle one oddball.  If there were more, I'd use another data structure.
        my_col_map["Formula"] = "membership_type"
    return my_col_map

In [5]:
trip_data_files=glob.glob('../data/Mobi*.*')

In [6]:
trip_data_raw=pd.DataFrame(columns=['departure',
                                   'return',
                                   'bike',
                                   'departure_station',
                                   'return_station',
                                   'membership_type',
                                   'covered_distance',
                                   'duration',
                                   'departure_temperature',
                                   'return_temperature'])

In [7]:
%%time
#Reading from large Excel files: NOT speedy.
for filename in trip_data_files:
    my_df=pd.DataFrame()
    if re.search('xls',filename):
        my_df=pd.read_excel(filename)
    elif re.search('csv',filename):
        my_df=pd.read_csv(filename)
    my_df.rename(columns=xform_columns(my_df.columns),inplace=True)
    trip_data_raw=pd.concat([trip_data_raw,
                            my_df[['departure',
                                   'return',
                                   'bike',
                                   'departure_station',
                                   'return_station',
                                   'membership_type',
                                   'covered_distance',
                                   'duration',
                                   'departure_temperature',
                                   'return_temperature']]],
                           ignore_index=1)

CPU times: user 3min 6s, sys: 7.4 s, total: 3min 14s
Wall time: 3min 17s


## Sunrise/sunset data import
Sunrise/sunset timing doesn't vary year-over-year at a level of granularity that matters to us (hourly).  This means that as long as we capture all the possible dates, we only need one year of data.

In [8]:
# The format of this file is a bit funky, and as fun as it would be to try to get
# get pandas to infer the headers from this 3-line monstrosity, we're 
# on a slightly different mission.
#
#        Twilight Start                      Twilight End      Hours of      Local Sidereal Time at 
#        --------------  Sun   Local  Sun   --------------   Illumination   00:00 PST 
#  Date  Nautical Civil  Rise  Noon   Set   Civil Nautical   Day Sky Total  hh:mm:ss 

sun_data_raw=pd.read_fwf('../data/sun_2019.txt',skiprows=17,skipfooter=1).dropna()

In [165]:
pretty_shape(sun_data_raw,"Sunrise/sunset data")
display(sun_data_raw.head())


Sunrise/sunset data:


,rows,cols
count,365,10


,Date,Nautical,Civil,Rise,Noon,Set,Civil Nautical,Day,Sky Total,hh:mm:ss
1,Jan 1,6:50,7:30,8:08,12:16,16:25,17:02 17:43,8.28,1.24 9.53,6:30:16
2,Jan 2,6:50,7:30,8:08,12:17,16:26,17:03 17:44,8.30,1.24 9.54,6:34:13
3,Jan 3,6:50,7:30,8:07,12:17,16:27,17:04 17:44,8.32,1.24 9.56,6:38:10
4,Jan 4,6:50,7:30,8:07,12:17,16:28,17:05 17:45,8.34,1.24 9.58,6:42:06
5,Jan 5,6:50,7:30,8:07,12:18,16:29,17:06 17:46,8.36,1.24 9.60,6:46:03


# Data cleanup and first-pass exploration

## Trips

In [166]:
pretty_shape(trip_data_raw,"Trip data")


Trip data:


,rows,cols
count,2258816,10


In [11]:
trip_data_raw.isna().sum()

departure                   0
return                    114
bike                     1109
departure_station           9
return_station            188
membership_type           416
covered_distance            0
duration                    0
departure_temperature       0
return_temperature          0
dtype: int64

In [12]:
display_embiggen(trip_data_raw['return_station'].value_counts().sort_index(),300)

0001 10th & Cambie                                                   27302
0002 Burrard Station                                                 16224
0004 Yaletown-Roundhouse Station                                     26814
0005 Dunsmuir & Beatty                                               26617
0006 Olympic Village Station                                         20617
0007 12th & Yukon                                                     2474
0008 8th & Ash                                                       15355
0009 Spyglass & Seawall                                              13073
0010 Stamp's Landing                                                 16734
0011 Ontario & Seawall                                               45431
0012 Dunsmuir & Richards                                             23559
0014 Canada Place                                                    31992
0015 Granville & Georgia                                             15512
0016 Pender & Burrard    

In [13]:
trip_data_raw['departure_station'].value_counts().index.size

238

In [14]:
#Looking at the weird ones...
my_re="\d+-"
trip_data_raw.fillna('none')\
            .query(f'return_station.str.contains("{my_re}",case=False)',engine="python")['return_station']\
            .value_counts()

35-0601 Olympic Village Station            7207
73-0151 Pine & 4th                         2230
137-0188 Granville Island Public Market    1614
76-0067 Cypress & 7th                       771
41-0075 10th & Arbutus                      306
222-0999.4 Smoove_test_4                      7
223-0999.5 Smoove_test_5                      1
Name: return_station, dtype: int64

In [15]:
# Easy cleanup: Drop trips with neither a departure nor return station
trip_data_raw=trip_data_raw.dropna(subset=['return_station','departure_station'])

In [16]:
# Cleaning up station info.  Comes in two parts:
# 1. Remove the leading digits from tho oddball stations 
# 2. Remove the text names of the stations 

# This will help aggregation later, since we have station IDs with 
# inconsistent text descriptions (e.g. 0051), and some legacy ID
# conventions (e.g. "35-0601 Olympic Village Station" instead of 0601)

station_regex=re.compile('(\d+-)*(\d+)(\s.*)')

test_df=trip_data_raw.head().copy()
test_df[['departure_station','return_station']]\
       .replace(station_regex,r'\2',regex=True)


,departure_station,return_station
0,0012,0041
1,0187,0048
2,0248,0244
3,0030,0174
4,0093,0032


In [167]:
# Before
pretty_shape(trip_data_raw,"Before station cleanup")
trip_data_raw[['return_station','departure_station']].head()


Before stations cleanup:


,rows,cols
count,2258816,10


,return_station,departure_station
0,0012,0041
1,0187,0048
2,0248,0244
3,0030,0174
4,0093,0032


In [18]:
# Update return and departure stations for all trips
trip_data_raw[['return_station','departure_station']]=\
    trip_data_raw[['departure_station','return_station']]\
                 .replace(station_regex,r'\2',regex=True)

In [168]:
#After
pretty_shape(trip_data_raw,"After station cleanup")
trip_data_raw[['return_station','departure_station']].head()


After stations cleanup:


,rows,cols
count,2258816,10


,return_station,departure_station
0,0012,0041
1,0187,0048
2,0248,0244
3,0030,0174
4,0093,0032


In [20]:
# We need to drop station returns or departures for the "admin" station IDs 
# (workshop, events, etc). IDs pulled from the list above
drop_stations=["0980","0981","0982",
               "0990","0991","0992","0993","0994","0995","0996","0997","0998",
               "0999","1000","9999","222-0999.4"]
qstr=f'return_station in {drop_stations} | departure_station in {drop_stations}'

trip_data_raw.drop(trip_data_raw.query(qstr).index,inplace=True)

In [21]:
pretty_shape(trip_data_raw,"After dropping stations")

(2258816, 10)

In [22]:
# Still have some nulls (bike ids and memberships), but none that we care about for the aggregation
trip_data_raw.isna().sum()

departure                   0
return                      0
bike                     1059
departure_station           0
return_station              0
membership_type           252
covered_distance            0
duration                    0
departure_temperature       0
return_temperature          0
dtype: int64

Checking out the time variables (return and departure), there are some interesting things to see:

In [23]:
print('Departures:')
display(pd.to_datetime(trip_data_raw['departure']).describe())
print('\nReturns:')
display(pd.to_datetime(trip_data_raw['return']).describe())

Departures:


count                 2258816
unique                  49137
top       2019-08-06 17:00:00
freq                      588
first     2017-01-01 00:00:00
last      2020-03-01 00:00:00
Name: departure, dtype: object


Returns:


count                 2258816
unique                  48936
top       2019-07-08 18:00:00
freq                      607
first     2017-01-01 01:00:00
last      2020-03-02 09:00:00
Name: return, dtype: object

Not only do the unique counts vary, but there are too many unique hours to account for our time period (Jan 2017-Feb 2020 should be roughly 8k\*3=24k, not ~50k.

According to Mobi, their trip data has been summarized to the hour, in part to protect member anonymity.  Erm, yeah, about that:


In [24]:
trip_data_raw['departure'].value_counts().tail()

4/9/19 20:32    1
4/9/19 20:50    1
4/1/19 23:50    1
4/9/19 13:44    1
4/5/19 13:27    1
Name: departure, dtype: int64

So, how do things look if we make the datestamps look like they should?

In [25]:
pd.to_datetime(trip_data_raw['departure']).dt.floor('h').describe()

count                 2258816
unique                  26923
top       2019-08-06 17:00:00
freq                      588
first     2017-01-01 00:00:00
last      2020-03-01 00:00:00
Name: departure, dtype: object

That is _significantly_ better.  Whew.

In [26]:
def fix_dates(my_df,series_name):
    print(f'\nBefore ({series_name}):')
    display(my_df[series_name].describe())
    my_df[series_name]=pd.to_datetime(my_df[series_name]).dt.floor('h')
    print(f'\nAfter ({series_name}):')
    display(my_df[series_name].describe())

In [27]:
fix_dates(trip_data_raw,'departure')
fix_dates(trip_data_raw,'return')


Before (departure):


count            2258816
unique             49141
top       8/6/2019 17:00
freq                 588
Name: departure, dtype: object


After (departure):


count                 2258816
unique                  26923
top       2019-08-06 17:00:00
freq                      588
first     2017-01-01 00:00:00
last      2020-03-01 00:00:00
Name: departure, dtype: object


Before (return):


count            2258816
unique             48963
top       7/8/2019 18:00
freq                 607
Name: return, dtype: object


After (return):


count                 2258816
unique                  26927
top       2019-07-08 18:00:00
freq                      607
first     2017-01-01 01:00:00
last      2020-03-02 09:00:00
Name: return, dtype: object

## Weather

In [67]:
display(weather_data_raw.info())
display(weather_data_raw.isna().sum())
pretty_shape(weather_data_raw,"Raw weather data records")
display(weather_data_raw.Weather.fillna("none").value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27720 entries, 0 to 743
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Longitudex        27720 non-null  float64
 1   Latitudey         27720 non-null  float64
 2   StationName       27720 non-null  object 
 3   ClimateID         27720 non-null  int64  
 4   Timestamp         27720 non-null  object 
 5   Year              27720 non-null  int64  
 6   Month             27720 non-null  int64  
 7   Day               27720 non-null  int64  
 8   Time              27720 non-null  object 
 9   TempC             27714 non-null  float64
 10  TempFlag          0 non-null      float64
 11  DewPointTempC     27714 non-null  float64
 12  DewPointTempFlag  0 non-null      float64
 13  RelHum            27714 non-null  float64
 14  RelHumFlag        0 non-null      float64
 15  WindDir10sdeg     27679 non-null  float64
 16  WindDirFlag       0 non-null      float64


None

Longitudex              0
Latitudey               0
StationName             0
ClimateID               0
Timestamp               0
Year                    0
Month                   0
Day                     0
Time                    0
TempC                   6
TempFlag            27720
DewPointTempC           6
DewPointTempFlag    27720
RelHum                  6
RelHumFlag          27720
WindDir10sdeg          41
WindDirFlag         27720
WindSpdkmh              6
WindSpdFlag         27720
Visibilitykm            6
VisibilityFlag      27720
StnPresskPa             6
StnPressFlag        27720
Hmdx                26355
HmdxFlag            27720
WindChill           26385
WindChillFlag       27720
Weather             15305
dtype: int64

(27720, 28)

none                                            15305
Rain                                             2886
Cloudy                                           2588
Mostly Cloudy                                    1945
Mainly Clear                                     1931
Clear                                            1169
Rain,Fog                                          390
Rain Showers                                      328
Fog                                               303
Snow                                              218
Moderate Rain                                     121
Moderate Rain,Fog                                 116
Smoke                                              98
Drizzle,Fog                                        93
Drizzle                                            35
Rain,Drizzle,Fog                                   31
Rain,Snow                                          22
Snow,Fog                                           21
Moderate Snow               

In [68]:
#Looks like we didn't get these in order?
#Also need to convert Timestamp column to a timestamp
print(weather_data_raw['Timestamp'].describe())
print(weather_data_raw['Timestamp'].head(1))

count                27720
unique               27720
top       2019-09-17 08:00
freq                     1
Name: Timestamp, dtype: object
0    2017-10-01 00:00
Name: Timestamp, dtype: object


In [69]:
weather_data_raw['Timestamp']=\
    pd.to_datetime(weather_data_raw['Timestamp'])
weather_data_raw.sort_values('Timestamp',inplace=True)
print(weather_data_raw['Timestamp'].describe())

count                   27720
unique                  27720
top       2017-09-24 19:00:00
freq                        1
first     2017-01-01 00:00:00
last      2020-02-29 23:00:00
Name: Timestamp, dtype: object


In [70]:
# There are a lot of gaps in the reported weather data, but spot-checks suggest 
# that they're pretty consistent.  It could be that descriptive weather notes are 
# only recorded every few hours, or when a notable change has occurred.
gap_lengths = pd.Series(([len(list(group_iter)) for k, group_iter in it.groupby(weather_data_raw['Weather'].isnull()) if k]))
display(gap_lengths.value_counts())
#Thanks to the brilliant nerd who posted about this "finding the length of gaps"
#solution on stackoverflow: https://stackoverflow.com/questions/16857407/pandas-run-length-of-nan-holes

2    7289
1     718
5       1
4       1
dtype: int64

In [71]:
# Most of the gaps are 1 or two hours.  Filling forward should be reasonable.
weather_data_raw['Weather'].fillna(method='ffill',inplace=True)

In [72]:
#Did we get em all?
weather_data_raw.Weather.isna().sum()

1

In [73]:
# You gotta be kidding me.
weather_data_raw[weather_data_raw['Weather'].isna()]

,Longitudex,Latitudey,StationName,ClimateID,Timestamp,Year,Month,Day,Time,TempC,...,WindSpdFlag,Visibilitykm,VisibilityFlag,StnPresskPa,StnPressFlag,Hmdx,HmdxFlag,WindChill,WindChillFlag,Weather
0,-123.18,49.19,VANCOUVER INTL A,1108395,2017-01-01,2017,1,1,00:00,1.2,...,NaN,19.3,NaN,100.54,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# That makes sense. Fill-forward isn't going to catch the first record in the dataset.
weather_data_raw['Weather'].fillna(method='bfill',inplace=True)
weather_data_raw.Weather.isna().sum()

0

In [75]:
#Filling our remaining gaps. Not too many of them, thankfully.
weather_data_raw[weather_data_raw['TempC'].isna()]

,Longitudex,Latitudey,StationName,ClimateID,Timestamp,Year,Month,Day,Time,TempC,...,WindSpdFlag,Visibilitykm,VisibilityFlag,StnPresskPa,StnPressFlag,Hmdx,HmdxFlag,WindChill,WindChillFlag,Weather
22,-123.18,49.19,VANCOUVER INTL A,1108395,2017-02-01 22:00:00,2017,2,1,22:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mainly Clear
177,-123.18,49.19,VANCOUVER INTL A,1108395,2017-11-08 09:00:00,2017,11,8,09:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cloudy
140,-123.18,49.19,VANCOUVER INTL A,1108395,2018-07-06 20:00:00,2018,7,6,20:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mostly Cloudy
141,-123.18,49.19,VANCOUVER INTL A,1108395,2018-07-06 21:00:00,2018,7,6,21:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mostly Cloudy
142,-123.18,49.19,VANCOUVER INTL A,1108395,2018-07-06 22:00:00,2018,7,6,22:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mostly Cloudy
407,-123.18,49.19,VANCOUVER INTL A,1108395,2018-08-17 23:00:00,2018,8,17,23:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mainly Clear


In [76]:
# All of our remaining nulls are handled by fixing only 6 rows.  
# Opting for simple forward-fill here.  
weather_data_raw[['TempC','RelHum','WindSpdkmh']]=\
    weather_data_raw[['TempC','RelHum','WindSpdkmh']]\
                    .fillna(method='ffill')
weather_data_raw[weather_data_raw['TempC'].isna()]

,Longitudex,Latitudey,StationName,ClimateID,Timestamp,Year,Month,Day,Time,TempC,...,WindSpdFlag,Visibilitykm,VisibilityFlag,StnPresskPa,StnPressFlag,Hmdx,HmdxFlag,WindChill,WindChillFlag,Weather


## Sunrise/sunset
Our bounding hours are civil twilight.  If nothing else, this project has taught me that there are 3 types of twilight. Seems like overkill to me, but scientists gonna science.

In [38]:
# This is kind of ugly.
# I could have used a list comprehension, my Favoritest Python Thing Ever.
# But that would have been even uglier.
# Stand ye in awe of my restraint.

daytime_lookup_df=pd.DataFrame([],
                               columns=['daylight_start',
                                         'daylight_end'])
for yr in [2017,2018,2019,2020]:
    twi_strt=pd.to_datetime(sun_data_raw['Date'] + f' {yr} ' + sun_data_raw['Civil'])
    twi_end=pd.to_datetime(sun_data_raw['Date'] + f' {yr} ' + sun_data_raw['Civil Nautical']\
                                                                           .str.split(expand=True)[0])
    daytime_lookup_df = pd.concat([daytime_lookup_df,
                                   pd.DataFrame(data={'daylight_start':twi_strt,
                                                      'daylight_end': twi_end},
                                                )],
                                  ignore_index=True,
                                  axis=0
                                 )

daytime_lookup_df.head()  

,daylight_start,daylight_end
0,2017-01-01 07:30:00,2017-01-01 17:02:00
1,2017-01-02 07:30:00,2017-01-02 17:03:00
2,2017-01-03 07:30:00,2017-01-03 17:04:00
3,2017-01-04 07:30:00,2017-01-04 17:05:00
4,2017-01-05 07:30:00,2017-01-05 17:06:00


## Assembling dataset

In [77]:
#Define target dataframe
#Weather columns added later
station_summary_df = pd.DataFrame(columns=['Timestamp', #Drop after encoding
                                     'is_weekend',
                                     'is_holiday',
                                     'station_id',
                                     'is_daylight',
                                     'num_returns',
                                     'num_departures'])

In [78]:
# The notion of interpreting the demand at a given station as number of
# returns + number of departures means that we have summarize the dataframe 
# twice: to count returns and to count departures, so we move from:
# return | departure | return_station | departure_station
# (one row per trip, separate return/departure timestamps and stations)
# to:
# timestamp | station_id | num_returns | num_departures
#(one row per timestamp(hour) + station, count of departures and returns)
# There might be a more pythonic/pandastic/whatever way to do this, 
# but so help me, the only thing that springs to mind is merging two 
# group-bys, so guess what we're going to do?

departure_df = trip_data_raw.groupby(['departure','departure_station'])\
                            .size()\
                            .reset_index()\
                            .rename(columns={'departure' : 'Timestamp',
                                             'departure_station' : 'station_id',
                                              0 : 'num_departures'})

return_df = trip_data_raw.groupby(['return','return_station'])\
                         .size()\
                         .reset_index()\
                         .rename(columns={'return' : 'Timestamp',
                                          'return_station' : 'station_id',
                                           0 : 'num_returns'})

station_summary_df[['Timestamp','station_id','num_returns','num_departures']] =\
  pd.merge(departure_df,
           return_df,
           on=['Timestamp','station_id'],
           how='outer')\
    .fillna(0)\
    .astype({'num_returns' : 'int32','num_departures':'int32'})\
    .sort_values('Timestamp')

In [79]:
display(departure_df['Timestamp'].describe())
display(return_df['Timestamp'].describe())
display(station_summary_df['Timestamp'].describe())

count                 1066955
unique                  26923
top       2019-08-12 17:00:00
freq                      160
first     2017-01-01 00:00:00
last      2020-03-01 00:00:00
Name: Timestamp, dtype: object

count                 1074732
unique                  26927
top       2019-08-19 18:00:00
freq                      149
first     2017-01-01 01:00:00
last      2020-03-02 09:00:00
Name: Timestamp, dtype: object

count                 1574914
unique                  27088
top       2019-09-03 18:00:00
freq                      180
first     2017-01-01 00:00:00
last      2020-03-02 09:00:00
Name: Timestamp, dtype: object

In [169]:
pretty_shape(station_summary_df,"Station trip summary")


Station trip summary:


,rows,cols
count,1574887,13


In [81]:
station_summary_df =\
  pd.merge(station_summary_df,
           weather_data_raw[['Timestamp','TempC','RelHum','WindSpdkmh','Weather']],
           on=['Timestamp'],
           how='inner')
pretty_shape(station_summary_df,"Station trip summary")

(1574887, 11)

In [82]:
# We lost a few records there, but it's the handful from March 2020:
display(station_summary_df['Timestamp'].describe())

count                 1574887
unique                  27080
top       2019-09-03 18:00:00
freq                      180
first     2017-01-01 00:00:00
last      2020-02-29 23:00:00
Name: Timestamp, dtype: object

In [83]:
# Update whether the hour is in the daylight range for the day.
# Gluing the columns on to do the calculation is crude, but my attempts to
# iterate over the daylight lookup dataframe (via loops, lambdas, etc)
# performed abysmally.
# On the plus side, merge_asof() has to be one of the coolest pandas 
# things ever.
station_summary_df.sort_values('Timestamp',inplace=True)
station_summary_df=pd.merge_asof(station_summary_df, daytime_lookup_df,
                                 left_on=['Timestamp'],
                                 right_on=['daylight_start'],
                                 direction='nearest'
                                 )


In [84]:
station_summary_df['is_daylight'] =\
  station_summary_df['Timestamp'].between(station_summary_df.daylight_start,
                                          station_summary_df.daylight_end)\
                                 .astype('int')

In [85]:
# Populate is_weekend flag
station_summary_df['is_weekend'] = (station_summary_df['Timestamp'].dt.dayofweek > 4).astype('int')

In [86]:
station_summary_df

,Timestamp,is_weekend,is_holiday,station_id,is_daylight,num_returns,num_departures,TempC,RelHum,WindSpdkmh,Weather,daylight_start,daylight_end
0,2017-01-01 00:00:00,1,NaN,0069,0,1,0,1.2,89.0,8.0,Cloudy,2017-01-01 07:30:00,2017-01-01 17:02:00
1,2017-01-01 01:00:00,1,NaN,0064,0,1,0,0.9,89.0,2.0,Cloudy,2017-01-01 07:30:00,2017-01-01 17:02:00
2,2017-01-01 01:00:00,1,NaN,0148,0,1,0,0.9,89.0,2.0,Cloudy,2017-01-01 07:30:00,2017-01-01 17:02:00
3,2017-01-01 01:00:00,1,NaN,0030,0,0,1,0.9,89.0,2.0,Cloudy,2017-01-01 07:30:00,2017-01-01 17:02:00
4,2017-01-01 01:00:00,1,NaN,0067,0,0,1,0.9,89.0,2.0,Cloudy,2017-01-01 07:30:00,2017-01-01 17:02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574882,2020-02-29 23:00:00,1,NaN,0215,0,2,0,2.8,89.0,10.0,Clear,2020-03-01 06:23:00,2020-03-01 18:27:00
1574883,2020-02-29 23:00:00,1,NaN,0212,0,1,0,2.8,89.0,10.0,Clear,2020-03-01 06:23:00,2020-03-01 18:27:00
1574884,2020-02-29 23:00:00,1,NaN,0196,0,2,0,2.8,89.0,10.0,Clear,2020-03-01 06:23:00,2020-03-01 18:27:00
1574885,2020-02-29 23:00:00,1,NaN,0019,0,0,1,2.8,89.0,10.0,Clear,2020-03-01 06:23:00,2020-03-01 18:27:00


Create calendar for identifying holidays

In [87]:
# Rules derived from https://en.wikipedia.org/wiki/Public_holidays_in_Canada#British_Columbia
# ...and stolen from US holiday definitions where matching.

class BCStatHols(hol.AbstractHolidayCalendar):
    rules = [
        hol.Holiday("New Years Day", month=1, day=1, observance=hol.nearest_workday),
        hol.Holiday("Family Day", end_date=datetime(2017,12,31),month=2, day=1,offset=hol.DateOffset(weekday=hol.MO(2))),
        hol.Holiday("Family Day", start_date=datetime(2018,1,1),month=2, day=1,offset=hol.DateOffset(weekday=hol.MO(3))),
        hol.GoodFriday,
        hol.Holiday("Victoria Day", month=5, day=24,offset=hol.DateOffset(weekday=hol.MO(-1))), #DateOffSet inclusive, hence "first Mon before 5/25" represented as "before 5/24 here"
        hol.Holiday("Canada Day", month=7, day=1, observance=hol.nearest_workday),
        hol.Holiday("BC Day", month=8, day=1, offset=hol.DateOffset(weekday=hol.MO(1))),
        hol.USLaborDay,
        hol.Holiday("Thanksgiving Day", month=10, day=1, offset=hol.DateOffset(weekday=hol.MO(2))),
        hol.Holiday("Remembrance Day",month=11, day=11, observance=hol.nearest_workday),
        hol.Holiday("Christmas", month=12, day=25, observance=hol.nearest_workday),
    ]
    

In [88]:
# Validating calendar
my_cal=BCStatHols()
my_cal.holidays(start='01-01-2017',end='12-31-2020',return_name=True)

2017-01-02       New Years Day
2017-02-13          Family Day
2017-04-14         Good Friday
2017-05-22        Victoria Day
2017-06-30          Canada Day
2017-08-07              BC Day
2017-09-04           Labor Day
2017-10-09    Thanksgiving Day
2017-11-10     Remembrance Day
2017-12-25           Christmas
2018-01-01       New Years Day
2018-02-19          Family Day
2018-03-30         Good Friday
2018-05-21        Victoria Day
2018-07-02          Canada Day
2018-08-06              BC Day
2018-09-03           Labor Day
2018-10-08    Thanksgiving Day
2018-11-12     Remembrance Day
2018-12-25           Christmas
2019-01-01       New Years Day
2019-02-18          Family Day
2019-04-19         Good Friday
2019-05-20        Victoria Day
2019-07-01          Canada Day
2019-08-05              BC Day
2019-09-02           Labor Day
2019-10-14    Thanksgiving Day
2019-11-11     Remembrance Day
2019-12-25           Christmas
2020-01-01       New Years Day
2020-02-17          Family Day
2020-04-

In [89]:
# Setting is_holiday flag
station_summary_df['is_holiday'] =\
  station_summary_df['Timestamp'].dt.floor('d')\
                    .isin(my_cal.holidays())\
                    .astype(int)

Assigning weather catgories.  This will be an ordinal value:

0: Decent weather (clear, cloudy)  
1: Inconvenient but bearable weather: Rain/drizzle/fog  
2: "Why am I biking in this?" weather: Snow, ice, Heavy rain/storm.

In [170]:
#Time to do weather things
mild_reg=re.compile('Clear|Cloudy|Haze',flags=re.IGNORECASE)
meh_reg=re.compile('Rain|Fog|Drizzl|Smoke',flags=re.IGNORECASE)
omg_reg=re.compile('Shower|Snow|Ice|Hail|Freez|Heavy|storm',flags=re.IGNORECASE)

weather_rank_filters=[mild_reg, meh_reg, omg_reg]

#Walk through the filters in reverse; we want the most severe match.
def rank_weather(desc, filters):
    for score, regex in reversed(list(enumerate(filters))):
        if re.search(regex,desc):
            return score
    return desc #Need to keep old value so it shows in dataframe as needing correction

# Test the regex filters; anything not caught should have a non-numeric value.
test_phrases=['Heavy Rain,Moderate Hail,Fog', #Expect 2 (Heavy, Hail)
              'Drizzle,Fog', #Expect 1
              'wtf', #Expect wtf
              'Rain Showers', #Expect 2 (Showers)
              'Mostly Cloudy', #Expect 0
              'Mainly Clear'] #Expect 0

for desc in test_phrases:
    print(f'Phrase: {desc}  Score: {rank_weather(desc,weather_rank_filters)}')
        

Phrase: Heavy Rain,Moderate Hail,Fog  Score: 2
Phrase: Drizzle,Fog  Score: 1
Phrase: wtf  Score: wtf
Phrase: Rain Showers  Score: 2
Phrase: Mostly Cloudy  Score: 0
Phrase: Mainly Clear  Score: 0


In [91]:
station_summary_df['Weather'] =\
    station_summary_df['Weather']\
                      .apply(lambda x: rank_weather(x,weather_rank_filters))
station_summary_df['Weather'].value_counts()

In [92]:
#Double-check that all these merges didn't introduce nulls

station_summary_df.isna().sum()

Timestamp         0
is_weekend        0
is_holiday        0
station_id        0
is_daylight       0
num_returns       0
num_departures    0
TempC             0
RelHum            0
WindSpdkmh        0
Weather           0
daylight_start    0
daylight_end      0
dtype: int64

In [93]:
#Now we explode the thing.

model_data=station_summary_df.copy()

In [94]:
#Add features for parts of timestamp (day of week already covered)
model_data['Year']=model_data['Timestamp'].dt.year
model_data['Month']=model_data['Timestamp'].dt.month
model_data['Hour']=model_data['Timestamp'].dt.hour
model_data.drop(columns=['Timestamp','daylight_start','daylight_end'],inplace=True)
model_data.head().T

,0,1,2,3,4
is_weekend,1,1,1,1,1
is_holiday,0,0,0,0,0
station_id,0069,0064,0148,0030,0067
is_daylight,0,0,0,0,0
num_returns,1,1,1,0,0
num_departures,0,0,0,1,1
TempC,1.2,0.9,0.9,0.9,0.9
RelHum,89,89,89,89,89
WindSpdkmh,8,2,2,2,2
Weather,0,0,0,0,0


In [95]:
# Generate test-train split before modeling starts

model_ind=model_data.copy()

model_dep=model_ind[['num_departures','num_returns']].copy()

model_ind.drop(['num_departures','num_returns'],axis=1,inplace=True)

(model_train_ind,model_test_ind,model_train_dep,model_test_dep) =\
  train_test_split(model_ind,model_dep,test_size=0.2,random_state=42)

In [175]:
pretty_shape(model_data,"Unsplit model data")
pretty_shape(model_train_ind,"Train set X")
pretty_shape(model_train_dep,"Train set Y")
pretty_shape(model_test_ind,"Test set X")
pretty_shape(model_test_dep,"Test set Y")


Unsplit model data:


,rows,cols
count,1574887,13



Train set X:


,rows,cols
count,1259909,11



Train set Y:


,rows,cols
count,1259909,2



Test set X:


,rows,cols
count,314978,11



Test set Y:


,rows,cols
count,314978,2


In [98]:
#Write out the dataframes of interest, for future reference
data_dir='../data'
trip_data_raw.to_csv(f'{data_dir}/trip_data_raw.csv.zip',compression='zip',index=False)
weather_data_raw.to_csv(f'{data_dir}/weather_data_raw.csv',index=False)
sun_data_raw.to_csv(f'{data_dir}/sun_data_raw.csv',index=False)
daytime_lookup_df.to_csv(f'{data_dir}/daytime_lookup_df.csv',index=False)
station_summary_df.to_csv(f'{data_dir}/station_summary_df.csv.zip',compression='zip',index=False)
model_data.to_csv(f'{data_dir}/model_data.csv.zip',compression='zip',index=False)
model_train_dep.to_csv(f'{data_dir}/model_train_dep.csv.zip',compression='zip',index=False)
model_train_ind.to_csv(f'{data_dir}/model_train_ind.csv.zip',compression='zip',index=False)
model_test_dep.to_csv(f'{data_dir}/model_test_dep.csv.zip',compression='zip',index=False)
model_test_ind.to_csv(f'{data_dir}/model_test_ind.csv.zip',compression='zip',index=False)


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 16.2 µs


## Station clustering
After two modeling iterations with station-level data, training accuracy scores on our chosen models were not high enough to provide useful results.  We return to the main model data and aggregate it based on our station clusters established in the modeling notebook.

In [100]:
stations_df=pd.read_csv(f'{data_dir}/station_clusters.csv')

In [116]:
#running through data assembly again, summarizing on cluster_id instead of station_id
station_summary_clustered=station_summary_df.copy()
station_summary_clustered['station_id']=\
    station_summary_clustered['station_id'].astype('int')

In [177]:
pretty_shape(station_summary_clustered,
             "Starting point before summarizing by cluster")


Starting point before summarizing by cluster:


,rows,cols
count,231162,11


In [118]:
#Add station cluster number
station_summary_clustered=pd.merge(station_summary_clustered,stations_df[['station_id','station_cluster']],
                       how='inner',
                       on=['station_id'])
station_summary_clustered.head().T

In [137]:
station_summary_clustered.groupby(['Timestamp','station_cluster']).agg({'num_returns':'sum',
                                                                        'num_departures':'sum'}).reset_index()
station_summary_clustered.drop(columns=['station_id','daylight_start','daylight_end'],inplace=True)

In [141]:
station_summary_clustered=station_summary_clustered.groupby(['Timestamp','is_weekend','is_holiday','is_daylight','TempC',
                                  'RelHum','WindSpdkmh','Weather','station_cluster']).agg({'num_returns':'sum',
                                                                        'num_departures':'sum'}).reset_index()

In [179]:
pretty_shape(station_summary_clustered,
             "After aggregating by station cluster")


After aggregating by station cluster:


,rows,cols
count,231162,11


In [147]:
# A little bit of copypasta from the modeling notebook

def encode_clustered(my_df):
    my_df = pd.get_dummies(my_df,
                         prefix = { 'station_cluster' : 'stn_cluster'},
                         columns = ['station_cluster'])
    my_df['Year']=my_df['Timestamp'].dt.year
    my_df['Month']=my_df['Timestamp'].dt.month
    my_df['Hour']=my_df['Timestamp'].dt.hour
    my_df['is_overnight'] = ((my_df['Hour']>22)|(my_df['Hour']<6))\
                            .astype(pd.SparseDtype(int, fill_value=0))
    my_df['is_morning']   = my_df['Hour'].between(6,9)\
                            .astype(pd.SparseDtype(int, fill_value=0))
    my_df['is_midday']    = my_df['Hour'].between(10,14)\
                            .astype(pd.SparseDtype(int, fill_value=0))
    my_df['is_lateaft']   = my_df['Hour'].between(15,18)\
                            .astype(pd.SparseDtype(int, fill_value=0))
    my_df['is_evening']   = my_df['Hour'].between(19,22)\
                            .astype(pd.SparseDtype(int, fill_value=0))
    my_df['is_winter']    = ((my_df['Month'] == 12) | (my_df['Month'] < 3))\
                            .astype(pd.SparseDtype(int, fill_value=0))
    my_df['is_spring']    = my_df['Month'].between(3,5)\
                            .astype(pd.SparseDtype(int, fill_value=0))
    my_df['is_summer']    = my_df['Month'].between(6,8)\
                            .astype(pd.SparseDtype(int, fill_value=0))
    my_df['is_autumn']    = my_df['Month'].between(9,11)\
                            .astype(pd.SparseDtype(int, fill_value=0))
    my_df.drop(columns=['Hour','Year','Month','Timestamp'],inplace=True)
    return my_df

In [148]:
model_data_clustered=encode_clustered(station_summary_clustered.copy())

In [180]:
pretty_shape(model_data_clustered,"Data aggregated by cluster and re-encoded")


Data aggregated by cluster and re-encoded:


,rows,cols
count,231162,29


In [152]:
model_data_clustered.head().T

,0,1,2,3,4
is_weekend,1,1,1,1,1
is_holiday,0,0,0,0,0
is_daylight,0,0,0,0,0
TempC,1.2,0.9,1.2,1.2,1.2
RelHum,89,89,96,96,96
WindSpdkmh,8,2,4,4,4
Weather,0,0,0,0,0
num_returns,1,2,1,0,0
num_departures,0,1,0,1,2
stn_cluster_0,0,0,1,0,0


In [154]:
# Generate test-train split before modeling round 3 starts
# This is not a 100% clean approach, since we returned the to the dataset
# midway through the modeling exercise.  At this point, however, we haven't 
# Looked at the test set for the unclustered data, either. 

clu_model_ind=model_data_clustered.copy()

clu_model_dep=clu_model_ind[['num_departures','num_returns']].copy()

clu_model_ind.drop(['num_departures','num_returns'],axis=1,inplace=True)

(clu_model_train_ind,clu_model_test_ind,clu_model_train_dep,clu_model_test_dep) =\
  train_test_split(clu_model_ind,clu_model_dep,test_size=0.2,random_state=42)

pretty_shape(model_data_clustered,"Unsplit model data")
pretty_shape(clu_model_train_ind,"Train set X")
pretty_shape(clu_model_train_dep,"Train set Y")
pretty_shape(clu_model_test_ind,"Test set X")
pretty_shape(clu_model_test_dep,"Test set Y")


Unsplit model data shape: (231162, 29)

Train set X shape: (184929, 27)

Train set Y shape: (184929, 2)

Test set X shape: (46233, 27)

Test set Y shape: (46233, 2)


In [155]:
model_data_clustered.to_csv(f'{data_dir}/model_data_clustered.csv.zip',compression='zip',index=False)
clu_model_train_dep.to_csv(f'{data_dir}/clu_model_train_dep.csv.zip',compression='zip',index=False)
clu_model_train_ind.to_csv(f'{data_dir}/clu_model_train_ind.csv.zip',compression='zip',index=False)
clu_model_test_dep.to_csv(f'{data_dir}/clu_model_test_dep.csv.zip',compression='zip',index=False)
clu_model_test_ind.to_csv(f'{data_dir}/clu_model_test_ind.csv.zip',compression='zip',index=False)